In [12]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.2


In [13]:
!python -V

Python 3.11.0


In [14]:
import pickle
import pandas as pd

In [15]:
with open('../../models/intro_lm.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [16]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [20]:
year = 2023 
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [18]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [19]:
y_pred.std()

np.float64(1.1368683772161603e-13)

In [21]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predicted_duration': y_pred})
output_file = f'predictions_{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)